In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn import plotting
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.second_level import SecondLevelModel
from nilearn.image import concat_imgs, mean_img, index_img
from nilearn.reporting import make_glm_report
import matplotlib.pyplot as plt

In [43]:
# format:
# /oak/stanford/groups/russpold/data/network_grant/discovery_BIDS_20250402/derivatives/output_lev1_mni/sub-s03/nBack/indiv_contrasts
# sub-s03_ses-02_task-nBack_contrast-twoBack-oneBack_rtmodel-rt_centered_stat-effect-size.nii.gz
BASE_DIR = '/oak/stanford/groups/russpold/data/network_grant/discovery_BIDS_20250402/derivatives/'
SUBJECTS = ['sub-s03', 'sub-s10', 'sub-s19', 'sub-s29', 'sub-s43']
SESSIONS = ['ses-01', 'ses-02', 'ses-03', 'ses-04', 'ses-05', 'ses-06', 'ses-07', 'ses-08', 'ses-09']
TASK = "nBack"
LEVEL = 'output_lev1_mni'

# Lists to store contrast maps and design information
all_contrast_maps = []
session_maps = {} # to keep track of them per subj (to normalize the order)
design_matrix_rows = []

# Contrast of interest
contrast_name = 'twoBack-oneBack_rtmodel-rt_centered_stat-effect-size'

In [47]:
# save the first level contrast map files
for subject in SUBJECTS:
    overall_session_count = 0
    session_maps[subject] = {}
    
    for session in SESSIONS:
        # Path to contrast image from first-level analysis
        contrast_map_path = os.path.join(
            BASE_DIR, LEVEL, subject, TASK, 'indiv_contrasts', f'{subject}_{session}_task-{TASK}_contrast-{contrast_name}.nii.gz'
        )
        
        # Check if file exists before adding
        if os.path.exists(contrast_map_path):
            all_contrast_maps.append(contrast_map_path)
            
            session_maps[subject][overall_session_count] = contrast_map_path
            overall_session_count += 1

In [48]:
# print(len(all_contrast_maps))
# print(design_matrix_rows)
# print(session_maps)

# sort the contrast maps per objectively numbered session
session_contrast_maps = {}
session_design_rows = {}

for session in range(5):
    session_contrast_maps[session] = []
    session_design_rows[session] = []
    
    for subject in SUBJECTS:
        if subject in session_maps and session in session_maps[subject]:
            map = session_maps[subject][session]
            session_contrast_maps[session].append(map)
            session_design_rows[session].append({
                'subject': subject,
                # Add any subject-specific covariates here
                # 'age': subject_age[subject],
                # 'performance': n_back_performance[subject][session]
            })

# print(session_contrast_maps)
# print("-----")
# print(session_design_rows) # session 5 (4 in the array numbering) only has 3 participants

**Will now create second level models and visualize them session by session. Can do this all in a loop in the future but this is for visualization purposes.**